# KarabukWildfire2025 – Analiz Defteri

Tek defter akışı: GEE init, AOI, Sentinel‑2 ön işleme, NDVI/NBR, dNDVI/dNBR, şiddet ve folium haritalar.
Önkoşullar: `pip install -r requirements.txt` ve bir kez `earthengine authenticate`.


In [1]:
# Importlar
import os
from IPython.display import IFrame, display
import pandas as pd

from src.pipeline import run_pipeline
from src.utils import ee_init


In [2]:
# Earth Engine proje kimliği (opsiyonel) ve erken başlatma
ee_project = 'karabukwildfire2025'
print('EE_PROJECT =', ee_project or '(not set; you may be prompted to authenticate)')
try:
    ee_init(project=ee_project)
    print('Earth Engine initialized.')
except Exception as e:
    print('Earth Engine init error:', e)


EE_PROJECT = karabukwildfire2025
Earth Engine initialized.


In [3]:
# Parametreler
pre_start, pre_end = '2025-07-10', '2025-07-25'
post_start, post_end = '2025-07-26', '2025-08-10'
aoi_path = 'src/aoi.geojson'  # yoksa varsayılan Karabük bbox kullanılır
out_dir = 'results'

print('AOI exists? ', os.path.exists(aoi_path))
print('OUT dir   : ', out_dir)
pre_start, pre_end, post_start, post_end


AOI exists?  True
OUT dir   :  results


('2025-07-10', '2025-07-25', '2025-07-26', '2025-08-10')

In [4]:
# Çalıştır
outputs = run_pipeline(
    pre_start=pre_start, pre_end=pre_end,
    post_start=post_start, post_end=post_end,
    aoi_geojson=aoi_path, out_dir=out_dir,
    project=ee_project
)
outputs


{'pre_rgb_map': 'results\\pre_RGB_2025-07-10_2025-07-25.html',
 'post_rgb_map': 'results\\post_RGB_2025-07-26_2025-08-10.html',
 'pre_ndvi_map': 'results\\pre_NDVI.html',
 'post_ndvi_map': 'results\\post_NDVI.html',
 'pre_nbr_map': 'results\\pre_NBR.html',
 'post_nbr_map': 'results\\post_NBR.html',
 'dndvi_map': 'results\\dNDVI.html',
 'dnbr_map': 'results\\dNBR.html',
 'severity_map': 'results\\severity.html',
 'summary_csv': 'results\\summary_stats.csv'}

In [5]:
# Haritaları defter içinde görüntüle (opsiyonel)
from IPython.display import HTML

def show_html(path, height=600):
    try:
        display(IFrame(src=path, width='100%', height=height))
    except Exception:
        try:
            with open(path, 'r', encoding='utf-8') as f:
                html = f.read()
            display(HTML(html))
        except Exception as e:
            display(HTML(f"<div style='color:#a00'>Unable to display {path}: {e}</div>"))

show_html(outputs['pre_ndvi_map'], height=480)
show_html(outputs['post_ndvi_map'], height=480)
show_html(outputs['pre_nbr_map'], height=480)
show_html(outputs['post_nbr_map'], height=480)
show_html(outputs['dnbr_map'], height=600)
show_html(outputs['severity_map'], height=600)


In [6]:
# Özet istatistikler
pd.read_csv(outputs['summary_csv'])


,metric,value
0,pre_mean_NDVI,0.535408
1,post_mean_NDVI,0.473173
2,mean_dNDVI,-0.062235
3,mean_dNBR,0.072990


In [7]:
# Rapor PNG'leri (analiz katmanları + doğal renk)
from src.gee.aoi import get_aoi
from src.gee.preprocess import prepare_composite
from src.gee.indices import with_indices
from src.gee.change import compute_diffs, classify_dnbr
from src.visualize import export_report_pngs, export_truecolor_pngs
from IPython.display import Image, HTML, display
import os, base64

aoi = get_aoi(aoi_path)
pre = with_indices(prepare_composite(aoi, pre_start, pre_end))
post = with_indices(prepare_composite(aoi, post_start, post_end))
diffs = compute_diffs(pre, post)
severity = classify_dnbr(diffs['dNBR'])

pngs = export_report_pngs(pre=pre, post=post, diffs=diffs, severity=severity, aoi=aoi, out_dir=out_dir)
rgbs = export_truecolor_pngs(pre=pre, post=post, aoi=aoi, out_dir=out_dir)

order = [
  ('Pre RGB', rgbs.get('pre_rgb_png')),
  ('Post RGB', rgbs.get('post_rgb_png')),
  ('Pre NDVI', pngs.get('pre_ndvi_png')),
  ('Post NDVI', pngs.get('post_ndvi_png')),
  ('dNBR', pngs.get('dnbr_png')),
  ('Severity', pngs.get('severity_png')),
]

def show_image(path):
    try:
        display(Image(filename=path))
    except Exception:
        try:
            with open(path,'rb') as f:
                data = base64.b64encode(f.read()).decode('ascii')
            display(HTML(f"<img src='data:image/png;base64,{data}' style='max-width:100%;height:auto;'/>"))
        except Exception as e:
            display(HTML(f"<div style='color:#a00'>Unable to display {path}: {e}</div>"))

for title, p in order:
    if not p or not os.path.exists(p):
        continue
    display(HTML(f"<h4 style='margin:8px 0'>{title}</h4>"))
    show_image(p)
